In [1]:
#Import Required Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import time
from pprint import pprint
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier 


# Data A: Placement Decisions 

In [2]:
# Import 'Placement Decisions' Data  (Shared by GWP Team)
Placement_Data = pd.read_excel('/Users/p0s00gs/Desktop/GWP/NHM_5_Markets/Input_Data/Banding_Report_NHM.xlsx', sheet_name= 'NHM_Stores')
Placement_Data = Placement_Data.drop_duplicates()
Placement_Data = Placement_Data.reset_index(drop = True)
Placement_Data.shape


(902, 20)

In [3]:
# String Operations on 'Column Name'
Placement_Data.columns = Placement_Data.columns.str.strip()
Placement_Data.columns = Placement_Data.columns.str.replace('-', '')
Placement_Data.columns = Placement_Data.columns.str.replace('  ', ' ')
Placement_Data.columns = Placement_Data.columns.str.replace(' ', '_')

In [4]:
# Keep only'FINALISED' Decision Data
Finalised_Data = Placement_Data[Placement_Data['STATUS_IN_COMPANION'] == 'FINALIZED']
Finalised_Data.shape

(793, 20)

In [5]:
#Drop unnecessary 4 Columns
Finalised_Data = Finalised_Data.drop(['STATUS_IN_COMPANION', 'SUGGESTED_ROLE3',
                                     'SUGGESTED_ROLE2', 'SUGGESTED_ROLE1'], 
                                     axis=1)
Finalised_Data.shape

(793, 16)

In [6]:
#Column Rename
Finalised_Data = Finalised_Data.rename(columns={'POSITION1_BANDING_COLOR': 'TL_OM_Band', 
                                                'POSITION2_BANDING_COLOR': 'AT_TL_Band',
                                                'POSITION1_BANDING_VALUE': 'TL_OM_Band_Val',
                                                'POSITION2_BANDING_VALUE': 'AT_TL_Band_Val'
                                               })
Finalised_Data.columns                               

Index(['DIVISION', 'REGION', 'MARKET', 'STORE_NBR', 'WIN_NBR', 'FIRST_NAME',
       'LAST_NAME', 'EVAL_SCORE', 'POSITION1', 'TL_OM_Band_Val', 'TL_OM_Band',
       'POSITION2', 'AT_TL_Band_Val', 'AT_TL_Band', 'CURRENT_ROLE',
       'PLACEMENT_ROLE'],
      dtype='object')

In [7]:
#Create Yes_No_Decision column 
def f_Yes_No_Decision(row):
    if row['PLACEMENT_ROLE'] == 'No Suggestion':
        val = 'Not_Placed'
    else:
        val = 'Placed'
    return val

In [8]:
#Call f_Yes_No_Decision function
Finalised_Data['Yes_No_Decision'] = Finalised_Data.apply(f_Yes_No_Decision, axis=1)

In [9]:
#Create AT_No_Decision column 
#To model 'AT or Not Placed as AT'

def f_AT_No_Decision(row):
    if row['PLACEMENT_ROLE'] == 'Fresh AT Prod':
        val = 'Placed_AT'
    elif row['PLACEMENT_ROLE'] == 'Front End AT Serve':
        val = 'Placed_AT'
    elif row['PLACEMENT_ROLE'] == 'Food & Consumable AT Avail':
        val = 'Placed_AT'
    elif row['PLACEMENT_ROLE'] == 'Fuel AT':
        val = 'Placed_AT'
    elif row['PLACEMENT_ROLE'] == 'Fresh AT Avail':
        val = 'Placed_AT'
    elif row['PLACEMENT_ROLE'] == 'O/N Receiving AT':
        val = 'Placed_AT'
    elif row['PLACEMENT_ROLE'] == 'Salesfloor Serve AT':
        val = 'Placed_AT'
    elif row['PLACEMENT_ROLE'] == 'Ecomm/OGP AT Avail':
        val = 'Placed_AT'
    else:
        val = 'Not_AT_Placed'
    return val

In [10]:
#Call f_AT_No_Decision function
Finalised_Data['AT_No_Decision'] = Finalised_Data.apply(f_AT_No_Decision, axis=1)

In [11]:
#Drop unnecessary 2 Columns
Finalised_Data = Finalised_Data.drop(['POSITION2', 'POSITION1'], axis=1)
Finalised_Data.shape

(793, 16)

In [12]:
# Create TL_No_Decision column 

def f_TL_No_Decision(row):
    if row['PLACEMENT_ROLE'] == 'Front End TL Serve':
        val = 'Placed_TL'
    elif row['PLACEMENT_ROLE'] == 'O/N TL Receiving':
        val = 'Placed_TL'
    elif row['PLACEMENT_ROLE'] == 'Fresh TL Avail':
        val = 'Placed_TL'
    elif row['PLACEMENT_ROLE'] == 'Food & Consumable TL Avail':
        val = 'Placed_TL'
    elif row['PLACEMENT_ROLE'] == 'Salesfloor Serve TL':
        val = 'Placed_TL'
    else:
        val = 'Not_TL_Placed'
    return val

In [13]:
#Call f_TL_No_Decision function
Finalised_Data['TL_No_Decision'] = Finalised_Data.apply(f_TL_No_Decision, axis=1)

In [14]:
#OM Model - Funciton to create OM_No_Decision column in df
def f_OM_No_Decision(row):
    if row['PLACEMENT_ROLE'] == 'Operations Manager':
        val = 'Placed_OM'
    else:
        val = 'Not_OM_Placed'
    return val

In [15]:
#Call f_OM_No_Decision function
Finalised_Data['OM_No_Decision'] = Finalised_Data.apply(f_OM_No_Decision, axis=1)

In [16]:
#Convert column datatype to 'Categorical'
Finalised_Data['TL_OM_Band'] = Finalised_Data.TL_OM_Band.astype('category')
Finalised_Data['AT_TL_Band'] = Finalised_Data.AT_TL_Band.astype('category')

#3 Y's (Target Variables) - choose Y carefully based on Business Problem & Data Available
Finalised_Data['Yes_No_Decision'] = Finalised_Data.Yes_No_Decision.astype('category')
Finalised_Data['AT_No_Decision'] = Finalised_Data.AT_No_Decision.astype('category')
Finalised_Data['TL_No_Decision'] = Finalised_Data.TL_No_Decision.astype('category')
Finalised_Data['OM_No_Decision'] = Finalised_Data.OM_No_Decision.astype('category')


In [17]:
# For multiclass classification - create All_Decisions column in same df
def AT_TL_OM_No_Decision(row):
    if row['Yes_No_Decision'] == 'Placed':
        if row['AT_No_Decision'] == 'Placed_AT':
            val = 'AT'
        elif row['TL_No_Decision'] == 'Placed_TL':
            val = 'TL'
        elif row['OM_No_Decision'] == 'Placed_OM':
            val = 'OM'
    else:
        val = 'Not_Placed'
    return val

In [18]:
#Call AT_TL_OM_No_Decision function
Finalised_Data['All_Decisions'] = Finalised_Data.apply(AT_TL_OM_No_Decision, axis=1)

# Data B: HRDW Associate (Alteryx Process)

In [19]:
#Load 'Associate Data' from HRDW
HRDW_Data = pd.read_excel('/Users/p0s00gs/Desktop/GWP/NHM_5_Markets/Input_Data/Alteryx_HRDW.xlsx', sheet_name= 'Sheet1')
HRDW_Data.shape


(953, 22)

In [20]:
#Drop nnecessary columns
HRDW_Data = HRDW_Data.drop(['BUSINESS_TYPE_DESC','STORE_TYPE_CODE',
                           'STORE_TYPE_DESC', 'BANNER_DESC', 'BUSINESS_TYPE_CODE',
                            'POSITION_DESC', 'JOB_DESC', 'JOB_FAMILY', 'NET_PAY', 'Store_Nbr',
                            'DriveTime', 'FIRST_NAME', 'LAST_NAME'
                           ], axis = 1)

In [21]:
#String Operations on Object type Column 'Values'
HRDW_Data['GENDER_CODE'] = HRDW_Data['GENDER_CODE'].str.strip()
HRDW_Data['ASC_TYPE_CODE'] = HRDW_Data['ASC_TYPE_CODE'].str.strip()
HRDW_Data['PAY_TYPE_CODE'] = HRDW_Data['PAY_TYPE_CODE'].str.strip()

In [22]:
#Few Associates have Drive Distance > 100 miles - maybe old address, etc in HRDW
HRDW_Data = HRDW_Data.fillna(100)

# Data C: Merge 'Placement Decisions' & 'HRDW' 

In [23]:
#Join Placement_Decisions_Data & HRDW_Data datasets for EDA
HRDW_Placements = pd.merge(Finalised_Data, HRDW_Data, on = 'WIN_NBR', how = 'left')
HRDW_Placements.shape

(793, 27)

In [24]:
#For Graph

def EEOC_placements(row):
    if row['DIVERSITY_CODE'] == 1:
        val = 'White or Caucasian'
    elif row['DIVERSITY_CODE'] == 2:
        val = 'African American or Black '
    elif row['DIVERSITY_CODE'] == 3:
        val = 'Hispanic or Latino'
    elif row['DIVERSITY_CODE'] == 5:
        val = 'Native American Indian/Alaskan'
    elif row['DIVERSITY_CODE'] == 6:
        val = 'Asian'
    elif row['DIVERSITY_CODE'] == 7:
        val = 'Other Pacific Islander/Native Hawaiian'
    elif row['DIVERSITY_CODE'] == 8:
        val = 'Two or More Races'
    else:
        val = 999
    return val

In [25]:
# Call EEOC_placements function
HRDW_Placements['Finalised_Candidates_By_EEOC'] = HRDW_Placements.apply(EEOC_placements, axis=1)


In [26]:
#Data type conversion
HRDW_Placements['Age_Y'] = HRDW_Placements['Age_Y'].astype(int)

In [27]:
#Create Seniority column using raw integer Age numbers
def Seniority(row):
    if row['Age_Y'] > 40:
        Seniority = 'Above 40'
    else:
        Seniority = 'Below/Equal to 40'
    return Seniority

In [28]:
# Call Seniority function
HRDW_Placements['Seniority'] = HRDW_Placements.apply(Seniority, axis=1)


In [29]:
#Column Rename
HRDW_Placements = HRDW_Placements.rename(columns={'stores_worked': 'Stores_Worked'})
                                                  

In [30]:
#For Graph
def plot_placements(row):
    if row['Yes_No_Decision'] == 'Not_Placed':
        val = 'Not_Placed'
    elif row['AT_No_Decision'] == 'Placed_AT':
        val = 'Placed_AT'
    elif row['TL_No_Decision'] == 'Placed_TL':
        val = 'Placed_TL'
    elif row['OM_No_Decision'] == 'Placed_OM':
        val = 'Placed_OM'
    else:
        val = 999
    return val


In [31]:
# Call plot_placements function
HRDW_Placements['Placement_Decisions'] = HRDW_Placements.apply(plot_placements, axis=1)

In [44]:
#On Average each store has 14 Placement Decisions to make
# 9 Placed & 5 Not Placed

# Data D: GWP Survey Results (UF & VR Aggrigated Data)

In [32]:
#Load simplified/aggregated data #SM View
GWP_Data = pd.read_excel('/Users/p0s00gs/Desktop/GWP/NHM_5_Markets/Input_Data/GWP_Survey_Results_NHM.xlsx', sheet_name= 'Data')
GWP_Data.shape

(774, 20)

In [33]:
# String Operations on 'Column Name'
GWP_Data.columns = GWP_Data.columns.str.strip()
GWP_Data.columns = GWP_Data.columns.str.replace('-', '')
GWP_Data.columns = GWP_Data.columns.str.replace('  ', ' ')
GWP_Data.columns = GWP_Data.columns.str.replace(' ', '_')

In [34]:
#Drop unnecessary columns 
GWP_Data = GWP_Data.drop(['First_Name','Last_Name', 'Store_Nbr', 'Market_Nbr', 
                           'UF_Store_Avg', 'UF_Mkt_Avg'], axis=1)


# Data E: Merge HRDW, Placement Decisions & GWP Survey 

In [35]:
#Join 2 datasets for EDA
GWP_HRDW_Placements = pd.merge(HRDW_Placements,GWP_Data, on = 'WIN_NBR', how = 'left')
GWP_HRDW_Placements.shape

(793, 43)

In [36]:
# UF_Overall can not be NULL
GWP_HRDW_Placements = GWP_HRDW_Placements[GWP_HRDW_Placements['UF_Overall'].notnull()]
GWP_HRDW_Placements.shape

(461, 43)

In [37]:
#Lower column names
GWP_HRDW_Placements.columns = map(str.lower, GWP_HRDW_Placements.columns)

In [38]:
#Drop unnecessary columns
GWP_HRDW_Placements = GWP_HRDW_Placements.drop(['first_name','last_name', 'current_role',
                                                'division', 'region', 'market',
                                               'store_nbr', 'placement_role', 
                                                'finalised_candidates_by_eeoc','diversity_code'            
                                                ], axis=1)
GWP_HRDW_Placements.shape

(461, 33)

In [39]:
#For Placed vs Not_Placed Model
def yes_no_to_binary(row):
    if row['yes_no_decision'] == 'Placed':
        val = 1
    else:
        val = 0
    return val

In [40]:
# Call yes_no_to_binary Function
GWP_HRDW_Placements['yes_no_decision'] = GWP_HRDW_Placements.apply(yes_no_to_binary, axis=1)


In [41]:
#For AT Model
def AT_no_to_binary(row):
    if row['at_no_decision'] == 'Placed_AT':
        val = 1
    else:
        val = 0
    return val

In [42]:
# Call AT_no_to_binary function
GWP_HRDW_Placements['at_no_decision'] = GWP_HRDW_Placements.apply(AT_no_to_binary, axis=1)


In [43]:
#For TL Model
def TL_no_to_binary(row):
    if row['tl_no_decision'] == 'Placed_TL':
        val = 1
    else:
        val = 0
    return val

In [44]:
#Call TL_no_to_binary function to create tl_no_decision column in df
GWP_HRDW_Placements['tl_no_decision'] = GWP_HRDW_Placements.apply(TL_no_to_binary, axis=1)


In [45]:
#For OM Model
def OM_no_to_binary(row):
    if row['om_no_decision'] == 'Placed_OM':
        val = 1
    else:
        val = 0
    return val

In [46]:
# Call OM_no_to_binary function
GWP_HRDW_Placements['om_no_decision'] = GWP_HRDW_Placements.apply(OM_no_to_binary, axis=1)


# Model Data Prep (Yes/No Decision)

In [47]:
yes_no_model = GWP_HRDW_Placements
yes_no_model.shape

(461, 33)

In [48]:
#Drop Unnecessary columns
del yes_no_model['placement_decisions']
del yes_no_model['vr_team_lead_score']
del yes_no_model['vr_academy_trainer_score']

In [50]:
#Fill NA for corerlation study - not used in Model 
yes_no_model['vr_team_lead_rank'].fillna('NA', inplace=True)
yes_no_model['vr_academy_trainer_rank'].fillna('NA', inplace=True)
yes_no_model['vr_fresh'].fillna('NA', inplace=True)
yes_no_model['vr_front_end'].fillna('NA', inplace=True)
yes_no_model['vr_sales_floor'].fillna('NA', inplace=True)
yes_no_model['vr_availability'].fillna('NA', inplace=True)


In [51]:
#Get dummies - convert categories to column
yes_no_model = pd.get_dummies(yes_no_model, columns=['gender_code'])
yes_no_model = pd.get_dummies(yes_no_model, columns=['tl_om_band'])
yes_no_model = pd.get_dummies(yes_no_model, columns=['at_tl_band'])
yes_no_model = pd.get_dummies(yes_no_model, columns=['pay_type_code'])
yes_no_model = pd.get_dummies(yes_no_model, columns=['asc_type_code'])
yes_no_model = pd.get_dummies(yes_no_model, columns=['seniority'])

yes_no_model = pd.get_dummies(yes_no_model, columns=['vr_team_lead_rank'])
yes_no_model = pd.get_dummies(yes_no_model, columns=['vr_academy_trainer_rank'])

yes_no_model = pd.get_dummies(yes_no_model, columns=['vr_fresh'])
yes_no_model = pd.get_dummies(yes_no_model, columns=['vr_front_end'])
yes_no_model = pd.get_dummies(yes_no_model, columns=['vr_sales_floor'])
yes_no_model = pd.get_dummies(yes_no_model, columns=['vr_availability'])


In [52]:
#Datatype Conversion
yes_no_model['tl_om_band_val'] = yes_no_model['tl_om_band_val'].astype(int)
yes_no_model['at_tl_band_val'] = yes_no_model['at_tl_band_val'].astype(int)

yes_no_model['eval_score'] = yes_no_model['eval_score'].astype(int)
yes_no_model['age_y'] = yes_no_model['age_y'].astype(float)
yes_no_model['total_wmt_tenure_m'] = yes_no_model['total_wmt_tenure_m'].astype(int)

In [53]:
#Correlation Study
corr = yes_no_model.corr()
corr.style.background_gradient(cmap='coolwarm')


In [54]:
#Function to get Correlation Significance

def get_corr_pValue(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns = df.columns)
    pvalues = dfcols.transpose().join(dfcols, how = 'outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(stats.pearsonr(df[r], df[c])[1],4)
    return pvalues
    

In [55]:
#Get p-values of correlation
get_corr_pValue(yes_no_model)

,win_nbr,eval_score,tl_om_band_val,at_tl_band_val,yes_no_decision,at_no_decision,tl_no_decision,om_no_decision,age_y,total_wmt_tenure_m,...,vr_front_end_NA,vr_front_end_Poor Fit,vr_sales_floor_Excellent Fit,vr_sales_floor_Good Fit,vr_sales_floor_NA,vr_sales_floor_Poor Fit,vr_availability_Excellent Fit,vr_availability_Good Fit,vr_availability_NA,vr_availability_Poor Fit
win_nbr,0,0.9827,0.0778,0.2811,0.3694,0.004,0.3295,0,0,0,...,0,0.1677,0.0473,0.198,0,0.0971,0.071,0.0331,0,0.1804
eval_score,0.9827,0,0,0,0,0.0534,0,0.5517,0.8382,0.9921,...,0.0046,0.1193,0.0308,0.1893,0.0046,0.9347,0.0378,0.38,0.0046,0.8229
tl_om_band_val,0.0778,0,0,0,0,0.0003,0,0,0.4954,0.0591,...,0.7696,0.2382,0.5738,0.5849,0.7696,0.6625,0.3856,0.1628,0.7696,0.7002
at_tl_band_val,0.2811,0,0,0,0,0.4991,0,0,0.5164,0.1331,...,0.9634,0.6057,0.586,0.4681,0.9634,0.8197,0.2389,0.0457,0.9634,0.8086
yes_no_decision,0.3694,0,0,0,0,0,0,0,0.0725,0.0289,...,0,0.6386,0.0697,0.0312,0,0.0977,0,0.7767,0,0.5059
at_no_decision,0.004,0.0534,0.0003,0.4991,0,0,0,0,0.2058,0,...,0,0.3682,0.0971,0.0117,0,0,0,0.1916,0,0.7275
tl_no_decision,0.3295,0,0,0,0,0,0,0,0.2331,0.8254,...,0,0.6436,0.0132,0.0012,0,0.5079,0.0013,0.4034,0,0.3204
om_no_decision,0,0.5517,0,0,0,0,0,0,0.4657,0,...,0,0.0092,0.0035,0,0,0.0043,0,0.0103,0,0.0828
age_y,0,0.8382,0.4954,0.5164,0.0725,0.2058,0.2331,0.4657,0,0,...,0.0296,0.9273,0.3281,0.5711,0.0296,0.2567,0.0395,0.9996,0.0296,0.867
total_wmt_tenure_m,0,0.9921,0.0591,0.1331,0.0289,0,0.8254,0,0,0,...,0,0.3028,0.0362,0.1338,0,0.01,0.0013,0.0875,0,0.2817


In [56]:
#Drop VR columns - Missing Data
yes_no_model = yes_no_model.drop(['vr_team_lead_rank_Excellent Fit',
       'vr_team_lead_rank_Good Fit', 'vr_team_lead_rank_NA',
       'vr_team_lead_rank_Poor Fit', 'vr_academy_trainer_rank_Excellent Fit',
       'vr_academy_trainer_rank_Good Fit', 'vr_academy_trainer_rank_NA',
       'vr_academy_trainer_rank_Poor Fit', 'vr_fresh_Excellent Fit',
       'vr_fresh_Good Fit', 'vr_fresh_NA', 'vr_fresh_Poor Fit',
       'vr_front_end_Excellent Fit', 'vr_front_end_Good Fit',
       'vr_front_end_NA', 'vr_front_end_Poor Fit',
       'vr_sales_floor_Excellent Fit', 'vr_sales_floor_Good Fit',
       'vr_sales_floor_NA', 'vr_sales_floor_Poor Fit',
       'vr_availability_Excellent Fit', 'vr_availability_Good Fit',
       'vr_availability_NA', 'vr_availability_Poor Fit'
                                    ], axis=1)


In [57]:
#Drop unnecessary columns
yes_no_model = yes_no_model.drop([ 'win_nbr', 'tl_om_band_val', 'at_tl_band_val',
                                  'yes_no_decision', 'at_no_decision', 'tl_no_decision',
                                   'all_decisions', 'drivedistancemiles',
                                  'gender_code_F', 'gender_code_M',
                                  'uf_overall', 'uf_inspire_and_motivate',
                                   'uf_operate_with_discipline', 'uf_agreement'
                                 ], axis=1)


# Modeling

In [58]:
#Check OM Placed Associates
c =  yes_no_model.groupby(['om_no_decision']).size().reset_index(name = '#Associates')
c

,om_no_decision,#Associates
0,0,407
1,1,54


In [59]:
# String Operations on 'Column Name'
yes_no_model.columns = yes_no_model.columns.str.replace(' ', '_')
yes_no_model.columns = yes_no_model.columns.str.replace('/', '_')

In [60]:
#Correlation Study on Model Features
corr = yes_no_model.corr()
corr.style.background_gradient(cmap='coolwarm')


,eval_score,om_no_decision,age_y,total_wmt_tenure_m,stores_worked,uf_grow_the_business,tl_om_band_Green,tl_om_band_Red,tl_om_band_Yellow,at_tl_band_Green,at_tl_band_Red,at_tl_band_Yellow,pay_type_code_H,pay_type_code_S,asc_type_code_F,asc_type_code_P,seniority_Above_40,seniority_Below_Equal_to_40
eval_score,1,-0.027792,-0.00953428,-0.000460093,-0.0663247,0.191427,0.325603,-0.30235,0.019537,0.363804,-0.350718,-0.126184,0.194508,-0.194508,0.0286296,-0.0286296,-0.0265956,0.0265956
om_no_decision,-0.027792,1,0.0340621,0.202708,0.409009,0.0974586,0.368239,-0.27711,-0.041476,0.317287,-0.140897,-0.222121,-0.642989,0.642989,0.0890516,-0.0890516,0.0241756,-0.0241756
age_y,-0.00953428,0.0340621,1,0.444318,0.330235,-0.0653322,-0.0196806,0.0198853,-0.00275971,-0.0103886,-0.00913351,0.016611,-0.0929196,0.0929196,0.112044,-0.112044,0.864433,-0.864433
total_wmt_tenure_m,-0.000460093,0.202708,0.444318,1,0.662381,0.0286175,0.121205,-0.0929457,-0.0119499,0.116367,-0.0165085,-0.105353,-0.328349,0.328349,0.139812,-0.139812,0.361906,-0.361906
stores_worked,-0.0663247,0.409009,0.330235,0.662381,1,-0.01371,0.159343,-0.0972207,-0.0401955,0.146912,-0.0149155,-0.137034,-0.576099,0.576099,0.13588,-0.13588,0.27075,-0.27075
uf_grow_the_business,0.191427,0.0974586,-0.0653322,0.0286175,-0.01371,1,0.151282,-0.283551,0.149369,0.193933,-0.18711,-0.0671611,-0.043591,0.043591,0.0357519,-0.0357519,-0.0772199,0.0772199
tl_om_band_Green,0.325603,0.368239,-0.0196806,0.121205,0.159343,0.151282,1,-0.410615,-0.447899,0.548404,-0.208779,-0.407523,-0.129884,0.129884,0.131955,-0.131955,-0.0538944,0.0538944
tl_om_band_Red,-0.30235,-0.27711,0.0198853,-0.0929457,-0.0972207,-0.283551,-0.410615,1,-0.63132,-0.48133,0.481696,0.154937,0.0530973,-0.0530973,-0.223791,0.223791,0.0332759,-0.0332759
tl_om_band_Yellow,0.019537,-0.041476,-0.00275971,-0.0119499,-0.0401955,0.149369,-0.447899,-0.63132,1,0.00553855,-0.29476,0.194686,0.0584058,-0.0584058,0.107209,-0.107209,0.0132098,-0.0132098
at_tl_band_Green,0.363804,0.317287,-0.0103886,0.116367,0.146912,0.193933,0.548404,-0.48133,0.00553855,1,-0.341609,-0.769665,-0.20311,0.20311,0.140101,-0.140101,-0.0253252,0.0253252


In [61]:
#Convert to category data type

yes_no_model['om_no_decision'] = yes_no_model.om_no_decision.astype('category')

yes_no_model['tl_om_band_Green'] = yes_no_model.tl_om_band_Green.astype('category')
yes_no_model['tl_om_band_Red'] = yes_no_model.tl_om_band_Red.astype('category')
yes_no_model['tl_om_band_Yellow'] = yes_no_model.tl_om_band_Yellow.astype('category')

yes_no_model['at_tl_band_Green'] = yes_no_model.at_tl_band_Green.astype('category')
yes_no_model['at_tl_band_Red'] = yes_no_model.at_tl_band_Red.astype('category')
yes_no_model['at_tl_band_Yellow'] = yes_no_model.at_tl_band_Yellow.astype('category')

yes_no_model['pay_type_code_H'] = yes_no_model.pay_type_code_H.astype('category')
yes_no_model['pay_type_code_S'] = yes_no_model.pay_type_code_S.astype('category')

yes_no_model['asc_type_code_F'] = yes_no_model.asc_type_code_F.astype('category')
yes_no_model['asc_type_code_P'] = yes_no_model.asc_type_code_P.astype('category')

yes_no_model['seniority_Above_40'] = yes_no_model.seniority_Above_40.astype('category')
yes_no_model['seniority_Below_Equal_to_40'] = yes_no_model.seniority_Below_Equal_to_40.astype('category')



In [62]:
#Dropped after model feature importance results

del yes_no_model['asc_type_code_P']
del yes_no_model['asc_type_code_F']

del yes_no_model['seniority_Below_Equal_to_40']
del yes_no_model['seniority_Above_40']

del yes_no_model['age_y']
del yes_no_model['stores_worked']

In [64]:
#Seperate Features & Labels
features = yes_no_model.iloc[:, yes_no_model.columns != 'om_no_decision']
labels = yes_no_model.iloc[:, yes_no_model.columns == 'om_no_decision']


In [65]:
#Split TRAIN & TEST Data
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    labels,
                                                    test_size = 0.13, 
                                                    random_state = 101
                                                    #stratify=True
                                                   )


print('lets TRAIN model on {} examples'.format(X_train.shape[0]))
print('lets TEST model on {} examples'.format(X_test.shape[0]))


lets TRAIN model on 401 examples
lets TEST model on 60 examples


In [66]:
#Function for printing 'Feature Importance' of different models

def best_worst_features (scores):
    scores = sorted(scores, reverse = True)
    
    print("The 5 best features selected by this method are :")
    for i in range(5):
        print(scores[i][1])


In [67]:
#Define a basic RF model with 100 trees 
RF = RandomForestClassifier(n_estimators=100, 
                            random_state=101,
                            class_weight="balanced",
                            n_jobs=-1)

In [68]:
#FIT model 
RF.fit(X_train, y_train)
#TEST data - Prediction & Accuracy
RF_pred = RF.predict(X_test)
#TEST Data Accuray using Baseline Model
print('TEST data accuracy is:',accuracy_score(y_test, RF_pred))


TEST data accuracy is: 0.9166666666666666


In [69]:
#TRAIN data Prediction & Accuracy
forest_XTrain_pred = RF.predict(X_train)
print('TRAIN data accuracy is:', accuracy_score(y_train, forest_XTrain_pred))
      

TRAIN data accuracy is: 1.0


In [71]:
#Get Feature Importance - Top 5 Features by Basline Model
pd.DataFrame(RF.feature_importances_,
             index=X_train.columns, columns=['Importance']).sort_values(
             by='Importance', ascending=False)[:5]

,Importance
pay_type_code_H,0.341848
pay_type_code_S,0.283283
at_tl_band_Green,0.082996
total_wmt_tenure_m,0.081478
tl_om_band_Red,0.067598


In [72]:
#Hyperparameter Optimization using Random Search

forest = RandomForestClassifier(n_jobs = -1)

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 150, num = 130)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', None, 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1,20, num = 18)]
# Minimum number of samples required to split a node
min_samples_split = [2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}


In [73]:
#Scoring Used = 'Classification Accuracy'
rf_random = RandomizedSearchCV(estimator = forest, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=101, n_jobs = -1, scoring='accuracy')
# Fit the random search model
rf_random.fit(X_train, y_train)


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   54.5s finished
/Users/p0s00gs/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80...15, 16, 17, 18, 20], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18]},
          pre_dis

In [74]:
#Best random model 
rf_random.best_estimator_ 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=17, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=14, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [75]:
# best combination of hyper-parameters
rf_random.best_params_

{'n_estimators': 14,
 'min_samples_split': 4,
 'max_features': 'auto',
 'max_depth': 17}

In [76]:
#Update Hyperparameters of RF
RF_RandomCV = RandomForestClassifier( n_estimators = 14,
                             min_samples_split = 4,
                             max_features = 'auto',
                             max_depth = 17,
                             random_state=101,
                             n_jobs= -1)

In [77]:
#FIT model using Best Hyper Parameters
RF_RandomCV.fit(X_train, y_train)
#TEST data - Prediction & Accuracy
RF_pred = RF_RandomCV.predict(X_test)
#TEST Data Accuracy 
print('TEST data accuracy after hyperparameter tuning is:', accuracy_score(y_test, RF_pred))


TEST data accuracy after hyperparameter tuning is: 0.9166666666666666


In [78]:
#Print Confusion Metrics
from sklearn.metrics import confusion_matrix
confusion_matrix(RF_pred, y_test)

array([[46,  1],
       [ 4,  9]])

In [79]:
#Print Classification Report
print(classification_report(y_test, RF_pred))

              precision    recall  f1-score   support

           0       0.98      0.92      0.95        50
           1       0.69      0.90      0.78        10

   micro avg       0.92      0.92      0.92        60
   macro avg       0.84      0.91      0.87        60
weighted avg       0.93      0.92      0.92        60



In [80]:
#Check Std Deviation using 10 fold cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(RF_RandomCV, X_train, y_train, cv=10, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.87804878 1.         0.95121951 0.92682927 0.95       0.975
 0.975      0.94871795 1.         0.94871795]
Mean: 0.9553533458411507
Standard Deviation: 0.034179017249137765


In [81]:
#TRAIN Data: Prediction & Accuracy
forest_XTrain_pred = RF_RandomCV.predict(X_train)
print('TRAIN data accuracy after hyperparameter tuning is:', accuracy_score(y_train, forest_XTrain_pred))


TRAIN data accuracy after hyperparameter tuning is: 0.9875311720698254


In [84]:
#Feature Importance - Top 5
pd.DataFrame(RF_RandomCV.feature_importances_,
             index=X_train.columns, columns=['Importance']).sort_values(
             by='Importance', ascending=False)[:5]

,Importance
pay_type_code_H,0.297082
pay_type_code_S,0.197774
total_wmt_tenure_m,0.101877
tl_om_band_Green,0.084640
tl_om_band_Red,0.083864


# Deploy Model to Calculate Placement Inconsistencies

In [85]:
#create subset df for model deployment
model_deploy_yes_no = GWP_HRDW_Placements[['eval_score','total_wmt_tenure_m',
                                           'uf_grow_the_business', 'tl_om_band', 'at_tl_band',
                                           'pay_type_code', 
                                           'win_nbr', 'om_no_decision']]
model_deploy_yes_no.shape

(461, 8)

In [86]:
#Handle Categorical Columns - Get Dummies
model_deploy_yes_no = pd.get_dummies(model_deploy_yes_no, columns=['tl_om_band'])
model_deploy_yes_no = pd.get_dummies(model_deploy_yes_no, columns=['at_tl_band'])
model_deploy_yes_no = pd.get_dummies(model_deploy_yes_no, columns=['pay_type_code'])


In [87]:
#String Operations on 'Column Name' Strings
model_deploy_yes_no.columns = model_deploy_yes_no.columns.str.replace(' ', '_')
model_deploy_yes_no.columns = model_deploy_yes_no.columns.str.replace('/', '_')

In [88]:
#Convert required columns to Category data type

model_deploy_yes_no['tl_om_band_Green'] = model_deploy_yes_no.tl_om_band_Green.astype('category')
model_deploy_yes_no['tl_om_band_Red'] = model_deploy_yes_no.tl_om_band_Red.astype('category')
model_deploy_yes_no['tl_om_band_Yellow'] = model_deploy_yes_no.tl_om_band_Yellow.astype('category')
model_deploy_yes_no['at_tl_band_Green'] = model_deploy_yes_no.at_tl_band_Green.astype('category')
model_deploy_yes_no['at_tl_band_Red'] = model_deploy_yes_no.at_tl_band_Red.astype('category')
model_deploy_yes_no['at_tl_band_Yellow'] = model_deploy_yes_no.at_tl_band_Yellow.astype('category')

model_deploy_yes_no['pay_type_code_H'] = model_deploy_yes_no.pay_type_code_H.astype('category')
model_deploy_yes_no['pay_type_code_S'] = model_deploy_yes_no.pay_type_code_S.astype('category')



In [89]:
test = model_deploy_yes_no

In [90]:
test =test.rename(columns={'win_nbr': 'WIN_NBR'})

In [91]:
test.set_index('WIN_NBR', inplace=True)

In [92]:
test_predict = test.drop(['om_no_decision'], axis = 1)

In [93]:
#Do Prediction & Check Accuracy on TRAIN dataset
RF_pred = RF_RandomCV.predict(X_train)
accuracy_score(y_train, RF_pred)

0.9875311720698254

In [94]:
#TEST data - Prediction & Accuracy
RF_pred = RF_RandomCV.predict(X_test)
accuracy_score(y_test, RF_pred)

0.9166666666666666

In [95]:
#Predict Probability
c = RF_RandomCV.predict_proba(test_predict)

In [96]:
type(c)

numpy.ndarray

In [97]:
predict_proba_dataset = pd.DataFrame({'Lower':c[:,0],'Upper':c[:,1]})

In [98]:
#Predict CLASS/Label
test_predict['Predicted'] = RF_RandomCV.predict(test_predict)

In [99]:
test_predict = test_predict['Predicted']

In [100]:
x = pd.merge(test, test_predict, left_index=True, right_index=True)
x.shape

(461, 13)

In [101]:
#Total Model Accuracy - Model Performance
print('Total Model Accuracy is:',accuracy_score(x['om_no_decision'], x['Predicted']))

Total Model Accuracy is: 0.9783080260303688


In [102]:
x['WIN_NBR'] = x.index

In [103]:
x = x.reset_index(drop=True)

# Finding Unbiased Variance, Standard Deviation &  Confidence Interval (using forestci package from MIT)

In [104]:
CI_X_train = test[['eval_score','total_wmt_tenure_m', 'uf_grow_the_business',
       'tl_om_band_Green', 'tl_om_band_Red', 'tl_om_band_Yellow',
       'at_tl_band_Green', 'at_tl_band_Red', 'at_tl_band_Yellow',
       'pay_type_code_H', 'pay_type_code_S']]

CI_X_train.shape

(461, 11)

In [105]:
import forestci as fci

In [294]:
fci

<module 'forestci' from '/Users/p0s00gs/anaconda3/lib/python3.7/site-packages/forestci/__init__.py'>

In [106]:
train_x = CI_X_train
test_x = CI_X_train

In [107]:
## Calculate unbiased variance 
decisions_V_IJ_unbiased = fci.random_forest_error(RF_RandomCV, train_x, test_x)

In [108]:
#Get Standard Deviation
std_dev = np.sqrt(decisions_V_IJ_unbiased)
std_dev.shape

(461,)

In [109]:
arr = np.array(decisions_V_IJ_unbiased)
df = pd.DataFrame(data=arr.flatten())
Unbiased_Variance_df = pd.DataFrame(data=arr.flatten())
Unbiased_Variance_df.shape

(461, 1)

In [110]:
arr = np.array(std_dev)
df = pd.DataFrame(data=arr.flatten())
std_dev_df = pd.DataFrame(data=arr.flatten())
std_dev_df.shape

(461, 1)

In [111]:
df_merged = Unbiased_Variance_df.merge(std_dev_df, how='left', left_index=True, right_index=True)
df_merged.shape


(461, 2)

In [112]:
df2_merged = df_merged.merge(predict_proba_dataset, how='left', left_index=True, right_index=True)
df2_merged.shape


(461, 4)

In [113]:
df2_merged =df2_merged.rename(columns={'0_x': 'Unbiased_Variance', '0_y': 'Std_Dev'})

In [114]:
df_Out = df2_merged.merge(x, how='left', left_index=True, right_index=True)
df_Out.shape


(461, 18)

In [115]:
df_Out =df_Out.rename(columns={'Lower': 'Prob_0', 'Upper': 'Prob_1'})

In [116]:
df_Out['Max_Prob_0_1'] = df_Out[["Prob_0", "Prob_1"]].max(axis=1)

In [117]:
df_Out['sigma'] = df_Out['Std_Dev'] / np.sqrt(378)

In [118]:
df_Out['CI_Lower'] = (df_Out['Max_Prob_0_1']) - (1.96*df_Out['sigma'])  

In [119]:
df_Out['CI_Upper'] = (df_Out['Max_Prob_0_1']) + (1.96*df_Out['sigma'])  

In [321]:
df_Out.to_csv("Presentation_NHM_OM_NO.csv", index=False)